# 1) Imports

In [2]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from geopy.distance import great_circle

# 2) Coleta dos Dados

## 2.1) Lista de municípios do Sertão

In [90]:
mun_ser = pd.read_excel(r'../Dados/Municipios/lista-1262municipios-semiarido-2017.xlsx', header = 2).iloc[1:]
mun_ser = mun_ser[['Cod IBGE','UF', 'Município']]
mun_ser.columns = ['IBGE7', 'UF', 'NOME']

mun_cord = pd.read_csv(r'../Dados/Municipios/municipios.csv')
mun_cord = mun_cord[['codigo_ibge', 'latitude', 'longitude']]
mun_cord.columns = ['IBGE7', 'LATITUDE', 'LONGITUDE']

mun_ser = mun_ser.merge(mun_cord, on ='IBGE7')

## 2.2) Dados de registros administrativos 

In [4]:
imputer = IterativeImputer(random_state=42)

metrics = [
    'ANO', 'IBGE7','DIST_EF_PUB','DIST_EM_PUB','TTREVA_EF_PUB','TTREVA_EM_PUB','IDEB_AI',
    'IDEB_AF','DOCSUP_EF_PUB','DOCSUP_EM_PUB','TXNASC7C','TXNBAIXOP','PINTERSAP','PINTERDRSAI',
    'REN_PIBPC_D','PDEFAGUA','PDEFESGOTO','PANALF15','PIND_POS','SNIS_PAGUA','POP_TOT'
]

reg_adm_mun = pd.read_excel(r'../Dados/Registros Administrativos/dados_registros_administrativos_total_2012_2017.xlsx', sheet_name = 'MUNICÍPIO')
reg_adm_mun = reg_adm_mun[reg_adm_mun['IBGE7'].isin(mun_ser['IBGE7'])]
reg_adm_mun = reg_adm_mun[metrics]
reg_adm_mun = reg_adm_mun.dropna(axis = 1, thresh = reg_adm_mun.shape[0]*0.5)

num_metrics = ['ANO', 'DIST_EF_PUB', 'DIST_EM_PUB', 'IDEB_AI',
               'IDEB_AF', 'DOCSUP_EF_PUB', 'DOCSUP_EM_PUB', 'TXNASC7C', 'TXNBAIXOP',
               'PINTERSAP', 'PINTERDRSAI', 'REN_PIBPC_D', 'PDEFAGUA', 'PDEFESGOTO',
               'PANALF15', 'PIND_POS', 'SNIS_PAGUA', 'POP_TOT']


reg_adm_mun[num_metrics] = imputer.fit_transform(reg_adm_mun[num_metrics])
reg_adm_mun = reg_adm_mun.drop(columns='ANO')
reg_adm_mun = reg_adm_mun.groupby(['IBGE7']).mean().reset_index()
reg_adm_mun

,IBGE7,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,TXNASC7C,TXNBAIXOP,PINTERSAP,PINTERDRSAI,REN_PIBPC_D,PDEFAGUA,PDEFESGOTO,PANALF15,PIND_POS,SNIS_PAGUA,POP_TOT
0,2100907,27.726667,41.880000,4.226667,3.533333,9.96,43.20,34.121838,5.630394,50.869073,20.391608,4.049333,65.428667,81.306667,15.752667,63.516000,34.414000,45225.2
1,2112209,28.480000,43.720000,4.513333,3.833333,41.80,69.74,35.199081,8.127748,22.069940,6.172664,6.352667,18.005333,27.246000,10.698000,50.869333,98.230000,164769.2
2,2200053,28.493333,31.313333,4.966667,4.126667,25.84,44.30,68.242635,7.409374,35.277390,12.931658,4.136000,88.346667,82.507333,16.216000,63.552000,97.856000,6930.6
3,2200251,32.426667,39.646667,4.200000,3.806667,41.52,54.34,64.334096,7.336248,39.498288,16.492665,4.198000,57.053333,77.899333,20.174000,67.692000,96.424000,7498.8
4,2200277,30.020000,41.266667,4.246667,3.580000,25.60,32.24,58.080359,4.717489,25.194699,8.940359,4.580000,33.862000,91.522667,20.756667,60.358667,99.824667,4986.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,3170651,11.586667,25.853333,5.773333,4.546667,50.80,69.50,90.793519,9.441182,13.290917,0.406842,5.865333,42.881333,93.562000,15.380000,38.703333,100.000000,4988.0
1257,3170800,19.133333,32.260000,5.726667,4.166667,75.00,75.34,78.559308,7.359871,17.821959,1.405139,10.833333,13.718667,84.821333,11.657333,20.580000,92.042000,38518.4
1258,3170909,13.366667,36.066667,6.333333,4.266667,70.74,64.96,74.494117,10.386963,31.067938,2.873772,5.084667,32.126667,88.424667,13.079333,61.648667,99.704000,19701.0
1259,3171030,18.213333,31.906667,5.080000,4.060000,60.92,87.26,76.540342,10.185719,17.376833,1.612218,6.050667,33.683333,78.057333,15.522667,39.580000,96.884000,9051.6


## 2.3) Dados meteorológicos 

In [8]:
est_ser = list(set(mun_ser['UF']))
est_met = pd.DataFrame()

num_columns = [
    'LATITUDE', 'LONGITUDE', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"
]

for year in [2020, 2021, 2022]:
    
    file_path = f'../Dados/INMET/INMET_{year}.csv'
    file = pd.read_csv(file_path)
    file = file[['Data', 'UF', 'ESTACAO', 'LATITUDE', 'LONGITUDE', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"]]
    file = file[file['UF'].isin(est_ser)]
    file = file.dropna(axis = 0, thresh = 6)
    file[num_columns] = imputer.fit_transform(file[num_columns])
    file = file.groupby(['Data', 'UF', 'ESTACAO','LATITUDE', 'LONGITUDE']).agg({'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'sum', "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": 'mean'}).reset_index()
    est_met = pd.concat([est_met, file], ignore_index=True)

# Dados das estações meteorologicas 
est_met = est_met.drop(columns = 'Data')
est_met = est_met.groupby(['ESTACAO', 'UF']).mean().reset_index()
est_met.columns = ['ESTACAO', 'UF', 'LATITUDE', 'LONGITUDE', 'PREC_MED', 'TEMP_MED']
est_met

,ESTACAO,UF,LATITUDE,LONGITUDE,PREC_MED,TEMP_MED
0,ABROLHOS,BA,-17.963016,-38.703288,1.948837,25.525335
1,ACARAU,CE,-3.121100,-40.087239,2.025126,26.889929
2,AGUAS VERMELHAS,MG,-15.751536,-41.457787,2.519409,21.264759
3,AIMORES,MG,-19.532776,-41.090828,3.517734,23.986615
4,ALMENARA,MG,-16.166736,-40.687764,2.681535,25.466457
...,...,...,...,...,...,...
211,VALENCA DO PIAUI,PI,-6.399241,-41.739967,3.027655,26.807054
212,VARGINHA,MG,-21.566389,-45.404167,2.902376,20.133951
213,VICOSA,MG,-20.762607,-42.864013,4.248950,19.907925
214,VITORIA DA CONQUISTA,BA,-14.886405,-40.801364,2.949141,20.508022


In [9]:
cidades = mun_ser.copy()

# Crie uma função para calcular a distância entre duas coordenadas (latitude, longitude)
def calcular_distancia(cidade, estacao):
    cidade_coord = (cidade['LATITUDE'], cidade['LONGITUDE'])
    estacao_coord = (estacao['LATITUDE'], estacao['LONGITUDE'])
    return great_circle(cidade_coord, estacao_coord).kilometers

# Loop para calcular a distância e encontrar a estação mais próxima para cada cidade
for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in est_met.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    estacao_mais_proxima = est_met.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'PREC_MED'] = estacao_mais_proxima['PREC_MED']
    cidades.at[index_cidade, 'TEMP_MED'] = estacao_mais_proxima['TEMP_MED']
    
cidades

,IBGE7,UF,NOME,LATITUDE,LONGITUDE,PREC_MED,TEMP_MED
0,2700300,AL,ARAPIRACA,-9.75487,-36.6615,2.263969,26.327873
1,2700706,AL,BATALHA,-9.67420,-37.1330,1.953488,27.754398
2,2700904,AL,BELO MONTE,-9.82272,-37.2770,1.953488,27.754398
3,2701209,AL,CACIMBINHAS,-9.40121,-36.9911,2.296316,25.157018
4,2701605,AL,CANAPI,-9.11932,-37.5967,1.516842,27.046445
...,...,...,...,...,...,...,...
1256,2806008,SE,RIBEIRÓPOLIS,-10.53570,-37.4380,0.984831,26.969830
1257,2807006,SE,SÃO MIGUEL DO ALEIXO,-10.38470,-37.3836,1.894041,25.452166
1258,2807105,SE,SIMÃO DIAS,-10.73870,-37.8097,2.499337,24.440877
1259,2807303,SE,TELHA,-10.20640,-36.8818,4.713677,26.562289


## 2.4) Dados sobre recursos hídricos

In [10]:
atl_irr = pd.read_excel(r'../Dados/ANA/Atlas Irrigacao - Area Atual e Potencial.xlsx',header = 6).iloc[6:]
atl_irr = atl_irr[['Código','Área Total Irrigada', 'AAI - Potencial Total', 'AAI - Potencial Efetivo']]
atl_irr = atl_irr[atl_irr['Código'].isin(mun_ser['IBGE7'])]
atl_irr.columns = ['IBGE7','AREA_IRRIGADA_TOT', 'AREA_IRRIGADA_POT', 'AREA_IRRIGADA_POT_E']
atl_irr

,IBGE7,AREA_IRRIGADA_TOT,AREA_IRRIGADA_POT,AREA_IRRIGADA_POT_E
465,2100907,56.000000,829.401141,3.140248
653,2112209,666.262426,4269.334204,0.000000
667,2200053,96.000000,0.000000,0.000000
670,2200251,0.000000,1240.907963,0.000000
671,2200277,2.000000,0.000000,0.000000
...,...,...,...,...
3076,3170651,353.204987,1106.772079,142.399238
3079,3170800,4133.645939,37599.860831,1881.799953
3080,3170909,197.000000,400.497303,1.268556
3082,3171030,4822.626564,295.487156,3.395124


## 2.4) IDH e Habitação

In [45]:
idh = pd.read_excel(r'../Dados/Censo/Censo_municipal_estadual_nacional.xlsx', sheet_name = 'MUN 91-00-10')
idh = idh[(idh['Codmun7'].isin(mun_ser['IBGE7']))&(idh['ANO']==2010)]
idh = idh[['Codmun7', 'IDHM', 'T_BANAGUA','T_DENS','T_LIXO','T_LUZ','AGUA_ESGOTO','PAREDE']]
idh.columns = ['IBGE7', 'IDH', 'T_BANAGUA','T_DENS','T_LIXO','T_LUZ','AGUA_ESGOTO','PAREDE']

In [88]:
idh

,IBGE7,IDH,T_BANAGUA,T_DENS,T_LIXO,T_LUZ,AGUA_ESGOTO,PAREDE
11594,2100907,0.521,31.94,42.14,71.79,95.54,23.23,33.06
11782,2112209,0.649,76.38,39.60,77.55,98.66,7.39,17.06
11796,2200053,0.528,19.28,36.52,99.22,98.99,23.32,0.53
11799,2200251,0.531,48.55,24.04,97.24,86.10,10.77,0.21
11800,2200277,0.585,72.12,34.60,97.43,98.34,3.89,0.00
...,...,...,...,...,...,...,...,...
14205,3170651,0.634,75.25,18.35,88.58,96.62,16.25,0.16
14208,3170800,0.666,85.76,26.52,97.24,99.13,9.12,0.63
14209,3170909,0.594,66.33,37.32,67.62,98.34,17.92,0.38
14211,3171030,0.584,59.85,40.46,95.64,95.35,16.28,4.68


## 2.5) Qualidade da água

In [87]:
ql_agua = pd.read_csv(r'../Dados/ANA/Indicadores de Qualidade da Agua.csv')
ql_agua = ql_agua[['CDESTACAO','CORPODAGUA', 'SGUF','LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']]
ql_agua = ql_agua[ql_agua['SGUF'].isin(est_ser)]
ql_agua = ql_agua.dropna(axis = 0, thresh = 3)

num_columns = ['LATITUDE', 'LONGITUDE', 'MED_2020','MED_2021']

ql_agua[num_columns] = imputer.fit_transform(ql_agua[num_columns])
ql_agua['QUAL_MED_AGUA'] = (ql_agua['MED_2021'] + ql_agua['MED_2020'])/2
ql_agua = ql_agua.reset_index().drop(columns='index')
ql_agua

C:\Users\User\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:796: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,CDESTACAO,CORPODAGUA,SGUF,LATITUDE,LONGITUDE,MED_2020,MED_2021,QUAL_MED_AGUA
0,BS062,Rio Paraíba do Sul,MG,-22.930640,-43.124280,65.931501,64.943603,65.437552
1,PJ001,Rio Jaguari,MG,-22.880830,-46.391110,60.230061,58.059279,59.144670
2,PJ024,Rio Jaguari,MG,-22.840280,-46.306390,67.659397,68.612205,68.135801
3,PJ021,Rio Jaguari,MG,-22.831390,-46.123330,66.596533,67.779856,67.188194
4,PJ009,Rio Camanducaia,MG,-22.763330,-46.226390,57.283336,56.068683,56.676009
...,...,...,...,...,...,...,...,...
1758,MRT-01,Martinópole,CE,-3.239127,-40.615051,83.223868,86.230660,84.727264
1759,TUC-01,Tucunduba,CE,-3.182074,-40.449379,86.245055,89.013832,87.629443
1760,GAN-01,Gangorra,CE,-3.170566,-40.864792,87.701505,89.208336,88.454921
1761,DI2-01,Diamantino II,CE,-3.154950,-40.363972,88.215524,89.800519,89.008021


In [86]:
dist_corpo_agua_mais_prox = []

# Loop para calcular a distância e encontrar a estação mais próxima para cada cidade
for index_cidade, cidade in cidades.iterrows():
    distancias = []
    for index_estacao, estacao in ql_agua.iterrows():
        distancia = calcular_distancia(cidade, estacao)
        distancias.append(distancia)
    indice_estacao_mais_proxima = distancias.index(min(distancias))
    dist_corpo_agua_mais_prox.append(min(distancias))
    estacao_mais_proxima = ql_agua.loc[indice_estacao_mais_proxima]
    cidades.at[index_cidade, 'QUAL_MED_AGUA'] = estacao_mais_proxima['QUAL_MED_AGUA']

cidades['DIST_CORPO_AGUA'] = dist_corpo_agua_mais_prox
cidades

,IBGE7,UF,NOME,LATITUDE,LONGITUDE,PREC_MED,TEMP_MED,QUAL_MED_AGUA,DIST_CORPO_AGUA
0,2700300,AL,ARAPIRACA,-9.75487,-36.6615,2.263969,26.327873,62.439167,42.035186
1,2700706,AL,BATALHA,-9.67420,-37.1330,1.953488,27.754398,65.300843,24.642419
2,2700904,AL,BELO MONTE,-9.82272,-37.2770,1.953488,27.754398,65.300843,3.297594
3,2701209,AL,CACIMBINHAS,-9.40121,-36.9911,2.296316,25.157018,65.300843,58.541833
4,2701605,AL,CANAPI,-9.11932,-37.5967,1.516842,27.046445,66.893107,68.942666
...,...,...,...,...,...,...,...,...,...
1256,2806008,SE,RIBEIRÓPOLIS,-10.53570,-37.4380,0.984831,26.969830,64.911294,1.388835
1257,2807006,SE,SÃO MIGUEL DO ALEIXO,-10.38470,-37.3836,1.894041,25.452166,64.911294,16.508752
1258,2807105,SE,SIMÃO DIAS,-10.73870,-37.8097,2.499337,24.440877,65.088090,2.685846
1259,2807303,SE,TELHA,-10.20640,-36.8818,4.713677,26.562289,64.553680,4.298665


## 2.6) População total em um raio de 30km

In [102]:
cidades = cidades.merge(reg_adm_mun, on = 'IBGE7', how = 'inner')

# Função para calcular a população total dentro de um raio de 30 km de um município
def calcular_populacao_total(municipio, municipios_df):
    populacao_total = municipio['POP_TOT']  # Inclui a população do município atual
    for _, row in municipios_df.iterrows():
        distancia = great_circle((municipio['LATITUDE'], municipio['LONGITUDE']), (row['LATITUDE'], row['LONGITUDE'])).kilometers
        if distancia <= 30:
            populacao_total += row['POP_TOT']
    return populacao_total

# Crie uma nova coluna 'POP_TOTAL_30KM' para armazenar a população total em um raio de 30 km
cidades['POP_TOTAL_30KM'] = 0

# Itere sobre cada município para calcular a população total
for index, municipio in cidades.iterrows():
    populacao_total_30km = calcular_populacao_total(municipio, cidades)
    cidades.at[index, 'POP_TOTAL_30KM'] = populacao_total_30km
cidades

,IBGE7,UF,NOME,LATITUDE,LONGITUDE,PREC_MED,TEMP_MED,QUAL_MED_AGUA,DIST_CORPO_AGUA,DIST_EF_PUB,...,PINTERSAP,PINTERDRSAI,REN_PIBPC_D,PDEFAGUA,PDEFESGOTO,PANALF15,PIND_POS,SNIS_PAGUA,POP_TOT,POP_TOTAL_30KM
0,2700300,AL,ARAPIRACA,-9.75487,-36.6615,2.263969,26.327873,62.439167,42.035186,28.233333,...,18.636148,6.815611,11.666000,25.090000,81.802667,17.200000,40.258000,97.486,230975.6,582087.0
1,2700706,AL,BATALHA,-9.67420,-37.1330,1.953488,27.754398,65.300843,24.642419,29.840000,...,30.860307,12.150951,5.309333,49.420667,78.492667,17.761333,65.395333,88.194,18487.6,120483.6
2,2700904,AL,BELO MONTE,-9.82272,-37.2770,1.953488,27.754398,65.300843,3.297594,35.793333,...,20.308137,6.723170,6.660667,63.498667,86.385333,25.276667,40.738667,91.050,6774.4,114803.4
3,2701209,AL,CACIMBINHAS,-9.40121,-36.9911,2.296316,25.157018,65.300843,58.541833,38.426667,...,17.231791,2.046380,5.011333,69.006667,58.758000,22.470667,51.732000,32.604,10815.6,141797.4
4,2701605,AL,CANAPI,-9.11932,-37.5967,1.516842,27.046445,66.893107,68.942666,29.126667,...,12.902533,1.653252,3.688667,73.516000,60.924000,25.327333,38.208000,94.054,17958.4,148162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,2806008,SE,RIBEIRÓPOLIS,-10.53570,-37.4380,0.984831,26.969830,64.911294,1.388835,30.700000,...,8.830588,0.186191,8.620000,31.824000,38.268000,10.623333,51.443333,96.058,18359.0,100748.8
1257,2807006,SE,SÃO MIGUEL DO ALEIXO,-10.38470,-37.3836,1.894041,25.452166,64.911294,16.508752,33.133333,...,16.568992,1.666673,6.674000,30.828000,45.944000,14.550667,46.762000,95.810,3901.6,127390.8
1258,2807105,SE,SIMÃO DIAS,-10.73870,-37.8097,2.499337,24.440877,65.088090,2.685846,32.926667,...,20.306567,0.782715,8.642667,47.557333,49.266000,15.964667,64.088000,97.866,40522.2,120552.4
1259,2807303,SE,TELHA,-10.20640,-36.8818,4.713677,26.562289,64.553680,4.298665,31.706667,...,12.372443,0.612512,6.741333,17.638667,23.266000,13.462000,61.825333,98.560,3169.6,81132.8


## 2.7) Tabela final

In [104]:
cidades = cidades.merge(atl_irr, on = 'IBGE7', how = 'inner')
cidades = cidades.merge(idh, on = 'IBGE7', how = 'inner')
cidades

In [106]:
cidades.to_csv(r'../Dados/Tabela_final/tabela_final.csv')